### Feedback robustness

In [1]:
import logging

import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

from learner.models import dnns
from learner.learner import Learner
from data_store.datasets import decoy_mnist, decoy_mnist_CE_augmented
from xil_methods.xil_loss import RRRLoss, RRRGradCamLoss, CDEPLoss, HINTLoss, RBRLoss
import util

# Hyperparameters
EPOCHS = 50
BATCH_SIZE = 256
LR = 0.001
WEIGHT_DECAY = 0.0001
SAVE_LAST = True
VERBOSE_AFTER_N_EPOCHS = 2
SEED = [1, 10, 100, 1000, 10000]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Using {DEVICE} device]")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


[Using cuda device]


Note: we here use wrong instead of arbitrary feedback

In [2]:
methods = ['RRR', 'RRRGradCAM', 'RBR', 'CDEP', 'HINT', 'CE']
for feed in ['wrong', 'incomplete']:
    for n in range(5):
        util.seed_all(SEED[n])
        for meth in methods:
            if meth == 'RRR':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRLoss(reg)
            elif meth == 'RBR':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1000000
                loss_fn = RBRLoss(reg)
            elif meth == 'RRRGradCAM':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRGradCamLoss(reg)
            elif meth == 'HINT':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, hint_expl=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = 10
                loss_fn = HINTLoss(reg, last_conv_specified=True, upsample=True)
            elif meth == 'CE':
                train_dataloader, test_dataloader = decoy_mnist_CE_augmented(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = None
                loss_fn = nn.CrossEntropyLoss()
            elif meth == 'CDEP':
                train_dataloader, test_dataloader = decoy_mnist(fmnist=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 2000000  
                loss_fn = CDEPLoss(reg)
            print(f'{meth}-{feed}-{n}')
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            modelname = f'Robustness-{feed}-DecoyFMNIST-CNN-{meth}--seed={SEED[n]}'
            learner = Learner(model, loss_fn, optimizer, DEVICE, modelname)
            learner.fit(train_dataloader, test_dataloader, EPOCHS, save_last=SAVE_LAST, verbose_after_n_epochs=VERBOSE_AFTER_N_EPOCHS)
            test_acc, _ = learner.score(test_dataloader, learner.base_criterion)

[Using Seed= 1]
Generating dataset...
RRR-wrong-0
Start training...
Epoch 2| accuracy: 90.8%, loss: 0.001169 | Test Error: Acc: 72.6%, Avg loss: 0.007025
Epoch 4| accuracy: 94.0%, loss: 0.000755 | Test Error: Acc: 69.6%, Avg loss: 0.007111
Epoch 6| accuracy: 95.5%, loss: 0.000577 | Test Error: Acc: 66.1%, Avg loss: 0.007212
Epoch 8| accuracy: 96.4%, loss: 0.000478 | Test Error: Acc: 64.5%, Avg loss: 0.007272
Epoch 10| accuracy: 97.1%, loss: 0.000393 | Test Error: Acc: 63.0%, Avg loss: 0.007321
Epoch 12| accuracy: 97.2%, loss: 0.000369 | Test Error: Acc: 61.4%, Avg loss: 0.007386
Epoch 14| accuracy: 97.4%, loss: 0.000341 | Test Error: Acc: 63.7%, Avg loss: 0.007286
Epoch 16| accuracy: 97.7%, loss: 0.000305 | Test Error: Acc: 60.2%, Avg loss: 0.007430
Epoch 18| accuracy: 98.1%, loss: 0.000260 | Test Error: Acc: 61.9%, Avg loss: 0.007353
Epoch 20| accuracy: 98.2%, loss: 0.000250 | Test Error: Acc: 60.1%, Avg loss: 0.007432
Epoch 22| accuracy: 98.2%, loss: 0.000256 | Test Error: Acc: 61.1%

NameError: name 'f' is not defined

In [ ]:
methods = ['RRR', 'RRRGradCAM', 'RBR', 'CDEP', 'HINT', 'CE']
for feed in ['wrong', 'incomplete']:
    for n in range(5):
        util.seed_all(SEED[n])
        for meth in methods:
            if meth == 'RRR':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 10
                loss_fn = RRRLoss(reg)
            elif meth == 'RBR':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 100000
                loss_fn = RBRLoss(reg)
            elif meth == 'RRRGradCAM':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1
                loss_fn = RRRGradCamLoss(reg)
            elif meth == 'HINT':
                train_dataloader, test_dataloader = decoy_mnist(hint_expl=True, device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = 100
                loss_fn = HINTLoss(reg, last_conv_specified=True, upsample=True)
            elif meth == 'CE':
                train_dataloader, test_dataloader = decoy_mnist_CE_augmented(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)
                reg = None
                loss_fn = nn.CrossEntropyLoss()
            elif meth == 'CDEP':
                train_dataloader, test_dataloader = decoy_mnist(device=DEVICE, batch_size=BATCH_SIZE, feedback=feed)      
                reg = 1000000  
                loss_fn = CDEPLoss(reg)
            print(f'{meth}-{feed}-{n}')
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            model = dnns.SimpleConvNet().to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            modelname = f'Robustness-{feed}-DecoyMNIST-CNN-{meth}--seed={SEED[n]}'
            learner = Learner(model, loss_fn, optimizer, DEVICE, modelname)
            learner.fit(train_dataloader, test_dataloader, EPOCHS, save_last=SAVE_LAST, verbose_after_n_epochs=VERBOSE_AFTER_N_EPOCHS)
            test_acc, _ = learner.score(test_dataloader, learner.base_criterion)

[Using Seed= 1]
Loading dataset from existing file!
RRR-wrong-0
Start training...
Epoch 2| accuracy: 97.8%, loss: 0.000337 | Test Error: Acc: 97.1%, Avg loss: 0.005992
Epoch 4| accuracy: 98.9%, loss: 0.000161 | Test Error: Acc: 97.2%, Avg loss: 0.005973
Epoch 6| accuracy: 99.3%, loss: 0.000109 | Test Error: Acc: 96.3%, Avg loss: 0.006006
Epoch 8| accuracy: 99.6%, loss: 0.000067 | Test Error: Acc: 96.9%, Avg loss: 0.005976
Epoch 10| accuracy: 99.6%, loss: 0.000055 | Test Error: Acc: 94.7%, Avg loss: 0.006060
Epoch 12| accuracy: 99.7%, loss: 0.000041 | Test Error: Acc: 96.2%, Avg loss: 0.006004
Epoch 14| accuracy: 99.7%, loss: 0.000044 | Test Error: Acc: 94.4%, Avg loss: 0.006074
Epoch 16| accuracy: 99.9%, loss: 0.000022 | Test Error: Acc: 91.1%, Avg loss: 0.006202
Epoch 18| accuracy: 99.7%, loss: 0.000051 | Test Error: Acc: 91.3%, Avg loss: 0.006196
Epoch 20| accuracy: 99.8%, loss: 0.000031 | Test Error: Acc: 89.8%, Avg loss: 0.006256
Epoch 22| accuracy: 99.7%, loss: 0.000044 | Test Err